In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [4]:
Nx = 2048
Ny = 2048
phi = np.fromfile("phi_final.bin", dtype=np.float32).reshape((-1, Nx, Ny))
phi.shape

(100, 2048, 2048)

In [5]:
# Animate a scalar field phi[t, x, y] with shape (T, Nx, Ny) in a Jupyter notebook cell

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

def animate_phi(phi, fps=20, cmap="bwr", vmin=None, vmax=None, interval=None,
                repeat=True, save_path=None, dpi=120, origin="lower", title=None):
    """
    Show (and optionally save) an animation of phi with shape (T, Nx, Ny).

    Parameters
    ----------
    phi : np.ndarray, shape (T, Nx, Ny)
    fps : int            Frames per second for display/saving
    cmap : str           Matplotlib colormap
    vmin, vmax : float   Color limits; if None, use symmetric around max(|phi|)
    interval : int       Delay between frames in ms; if None, computed from fps
    repeat : bool        Loop animation
    save_path : str      If endswith('.mp4') or '.gif', will save to that file
    dpi : int            Output DPI for saving
    origin : str         'lower' (typical for fields) or 'upper'
    title : str          Optional title on the plot

    Returns
    -------
    HTML object for inline display (also shows automatically in notebooks).
    """
    assert phi.ndim == 3, "phi must be (T, Nx, Ny)"
    T, Nx, Ny = phi.shape

    # imshow expects (Ny, Nx); our frame is (Nx, Ny) → transpose
    # compute color limits
    if vmin is None or vmax is None:
        m = np.nanmax(np.abs(phi))
        vmin = -m if vmin is None else vmin
        vmax =  m if vmax is None else vmax

    if interval is None:
        interval = int(1000 / fps)

    fig, ax = plt.subplots(figsize=(6, 5))
    im = ax.imshow(phi[0].T, cmap=cmap, vmin=vmin, vmax=vmax, origin=origin, animated=True)
    cbar = plt.colorbar(im, ax=ax, pad=0.02, fraction=0.046)
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    if title is None:
        title = "t = 0 / {}".format(T-1)
    tt = ax.set_title(title)

    def update(frame_idx):
        im.set_array(phi[frame_idx].T)
        tt.set_text(f"t = {frame_idx} / {T-1}")
        return (im, tt)

    anim = FuncAnimation(fig, update, frames=T, interval=interval, blit=True, repeat=repeat)

    # Optional saving
    if save_path:
        if save_path.lower().endswith(".mp4"):
            from matplotlib.animation import FFMpegWriter
            anim.save(save_path, writer=FFMpegWriter(fps=fps), dpi=dpi)
        elif save_path.lower().endswith(".gif"):
            from matplotlib.animation import PillowWriter
            anim.save(save_path, writer=PillowWriter(fps=fps), dpi=dpi)
        else:
            print("Unsupported save_path extension; use .mp4 or .gif")

    plt.close(fig)  # prevent duplicate static figure output

# -------------------------
animate_phi(phi, fps=25, cmap="bwr", save_path="phasefield.mp4") 